# Imports

In [1]:
import sys 

import numpy as np 
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

import torch
import torchvision 
import torchvision.transforms as transforms

import pickle
import pandas as pd
import os

sys.path.append('../../Utils')

from SVC_Utils import *
from data_downloaders import *

# Load/Process TinyImageNet

In [16]:
get_tiny_imagenet('../../Datasets/')

train_dir = "../../Datasets/tiny-imagenet-200/train"
test_dir = "../../Datasets/tiny-imagenet-200/val"

# load training set 
trainset = torchvision.datasets.ImageFolder(train_dir, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=int(trainset.__len__()/2), shuffle=True, num_workers=2)
ftrainloader = torch.utils.data.DataLoader(trainset, batch_size=trainset.__len__(), shuffle=True, num_workers=2)

# load test set 
testset = torchvision.datasets.ImageFolder(test_dir, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=testset.__len__(), shuffle=False, num_workers=2)

Tiny ImageNet already downloaded.


In [17]:
traininputs, traintargets=load(trainloader)
testinputs, testtargets=load(testloader)
ftraininputs, ftraintargets=load(ftrainloader)

# Model Training

In [4]:
n_components=180
C_range=np.logspace(0,1,2)
gamma_range=np.logspace(-2,-1,2)

In [5]:
clfs=hp_grid(n_components=n_components, C_range=C_range, gamma_range=gamma_range)

In [6]:
fitted_clfs=train_grid(clfs, traininputs, traintargets)

Fitted:  1 / 4
Fitted:  2 / 4
Fitted:  3 / 4
Fitted:  4 / 4


# Model Testing/Evaluation

In [18]:
#Stores training and testing accuracies in matrices (Rows: C_range, Cols: gamma_range)

#train_accs=np.random.randn(len(C_range),len(gamma_range))
test_accs=np.random.randn(len(C_range),len(gamma_range))
test_preds=[]
k=0;

for i in range(len(C_range)):
    for j in range(len(gamma_range)):
        #train_accs[i,j]=predict_eval(fitted_clfs[k], traininputs, traintargets, training=True)[1]
        preds, test_accs[i,j]=predict_eval(fitted_clfs[k], testinputs, testtargets)
        test_preds.append(preds)
        k+=1

Testing Accuracy:  0.6
Testing Accuracy:  1.02
Testing Accuracy:  0.88
Testing Accuracy:  0.9199999999999999


In [22]:
idx=['C = 1','C = 10']
cols=['gamma = .01','gamma = .1']

trainacc_df=pd.DataFrame(data=train_accs, index=idx, columns=cols)
testacc_df=pd.DataFrame(data=test_accs, index=idx, columns=cols)

In [23]:
#training accuracy for C/gamma grid
trainacc_df.style.background_gradient(cmap='GnBu')

,gamma = .01,gamma = .1
C = 1,3.7,18.994
C = 10,12.824,92.246


In [24]:
#test accuracy for C/gamma grid
testacc_df.style.background_gradient(cmap='GnBu')

,gamma = .01,gamma = .1
C = 1,0.6,1.02
C = 10,0.88,0.92


# Save Models

In [25]:
maxacc, gen=maxacc_gen(test_accs, train_accs, clfs)

In [29]:
fn_max_acc = 'SVMTinyImageNet_maxacc_proba.pkl'
fn_gen = 'SVMTinyImageNet_gen_proba.pkl'

In [30]:
save_proba(fn_max_acc, maxacc, ftraininputs, ftraintargets)
save_proba(fn_gen, gen, ftraininputs, ftraintargets)